In [1]:
import pandas as pd
import censusdata

In [2]:

#CSV has official recreational marijuana tax info by the State of Colorado. Downloaded from https://data.colorado.gov/Revenue/Marijuana-Special-Tax-Revenue-by-City-in-Colorado/ehk3-i5tr
# Read our Colorado recreational weed tax data (2014-2018) into pandas

taxData_df = pd.read_csv("Resources/Marijuana_Special_Tax_Revenue_by_City_in_Colorado.csv")
#Cleaning the data by dropping irrelevant columns
taxData_df = taxData_df[["Municipality","Tax_Rev"]]
taxData_df.rename(columns={'Municipality': 'City'}, inplace=True)
taxData_df.dropna(inplace=True)
#Getting the sum for all cities
taxData_df = taxData_df.groupby('City').sum()
taxData_df.info

<bound method DataFrame.info of                                  Tax_Rev
City                                    
Adams County                   156987.00
Antonito                       218673.00
Aspen                          538223.78
Aurora                        4768785.84
Boulder                       2707634.81
Boulder County                 288675.11
Breckenridge                   585244.53
Carbondale                      97667.40
Central City                   107851.11
Combined Local Governments    5128176.42
Commerce City                   10348.00
Cortez                         677927.45
Crested Butte                  168560.76
De Beque                       131028.00
Denver                       20847472.16
Dillon                         210641.00
Dinosaur                        23008.00
Durango                       1290213.28
Eagle County                   285213.31
Edgewater                     1515789.35
Federal Heights                213505.00
Fort Collins             

In [7]:
taxData_df.count()


Tax_Rev    47
dtype: int64

In [38]:
# Okay now lets look at the census data
# We are using the censusdata library

In [3]:
mydata = censusdata.download('acs5', 2017,
                            censusdata.censusgeo([('state','08'), ('county', '*'),
                            ('county subdivision','*')]),
                            ['B19013_001E', 'B01003_001E', 'B01001_002E',
                             'B01001A_001E', 'B01001B_001E', 'B01001D_001E', 'B01001I_001E',
                             'B23025_004E', 'B01002A_001E', 'B09001_001E',
                             'B08136_001E'])
mydata['city'] = mydata.index
mydata = mydata.reset_index()

cities = []
clean = []
length = len(mydata['city'])
for i in range (0,length):
    cities.append(str(mydata['city'].tolist()[i]).split(',')[0])
for i in cities:
    end = (len(i.split(' ')))
    i = (i.split(' ')[:end-1])
    clean.append(" ".join(i))
    
census_df = pd.DataFrame({"City": [],"Income":[],"Population":[], "Male Pop (%)":[],
                   "White (%)":[],"Black (%)":[],"Asian (%)":[],"Hispanic (%)":[],
                   "Employed (%)":[], "Travel Time to Work (mins)":[],
                  "Median Age":[], "Age <18 (%)":[]})
census_df['City'] = clean
census_df['Income'] = mydata["B19013_001E"]
census_df['Population'] = mydata['B01003_001E']
census_df['Male Pop (%)'] = ((mydata['B01001_002E'])/(mydata['B01003_001E'])*100).round(2)
census_df["White (%)"] = ((mydata['B01001A_001E'])/(mydata['B01003_001E'])*100).round(2)
census_df["Black (%)"] = ((mydata['B01001B_001E'])/(mydata['B01003_001E'])*100).round(2)
census_df["Asian (%)"] = ((mydata['B01001D_001E'])/(mydata['B01003_001E'])*100).round(2)
census_df["Hispanic (%)"] = ((mydata['B01001I_001E'])/(mydata['B01003_001E'])*100).round(2)
census_df["Employed (%)"] = ((mydata['B23025_004E'])/(mydata['B01003_001E'])*100).round(2)
census_df['Median Age'] = mydata['B01002A_001E']
census_df['Age <18 (%)'] = ((mydata['B09001_001E'])/(mydata['B01003_001E'])*100).round(2)
census_df["Travel Time to Work (mins)"] = ((mydata['B08136_001E'])/(mydata['B23025_004E'])).round(0)

In [5]:
#pd.merge(df_a, df_b, on='subject_id', how='inner')

merged_df = pd.merge(census_df , taxData_df, on="City", how='inner')

In [6]:
merged_df

,City,Income,Population,Male Pop (%),White (%),Black (%),Asian (%),Hispanic (%),Employed (%),Travel Time to Work (mins),Median Age,Age <18 (%),Tax_Rev
0,Trinidad,39292,10819,49.58,84.37,0.32,0.90,47.81,45.77,NaN,43.9,19.05,1806684.35
1,Steamboat Springs,72132,17678,52.40,95.02,0.96,1.37,7.07,61.51,13.0,39.3,18.15,438511.65
2,Breckenridge,76701,17459,52.41,95.11,0.17,0.61,3.16,64.28,12.0,39.2,14.65,585244.53
3,Idaho Springs,71172,6754,50.33,93.75,1.01,0.98,4.53,56.11,NaN,48.3,17.56,274746.49
4,Pueblo,36840,115330,49.18,73.37,2.85,0.90,50.39,39.22,18.0,42.0,22.72,267145.00
5,De Beque,55000,901,50.50,98.89,0.00,0.00,6.10,41.29,NaN,37.4,30.97,131028.00
6,Antonito,25972,1807,52.85,89.71,1.49,0.00,86.44,32.87,NaN,45.2,21.69,218673.00
7,Commerce City,43133,32427,50.18,82.10,5.46,0.30,66.47,41.06,NaN,30.6,32.92,10348.00
8,Leadville,46397,6357,52.79,88.74,0.14,0.76,26.93,57.42,NaN,41.0,14.54,95791.00
9,Durango,63468,33437,49.59,89.78,0.57,0.62,11.42,55.12,NaN,39.0,17.94,1290213.28
